In [133]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
main_dir = os.getenv('main_dir')

In [134]:
import pandas as pd
import datetime as dt

In [135]:
data_dir = main_dir+'/data/data/'
train_file = data_dir+'train.csv'
val_file = data_dir+'validation.csv'
test_file = data_dir+'test.csv'

In [136]:
train = pd.read_csv(train_file)
train

,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Expected_checkin,Expected_checkout,...,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion,Discount_Rate,Room_Rate
0,39428300,F,40,Latino,Grad,<25K,North,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,No Deposit,Online,Yes,Check-In,Yes,10,218
1,77491756,F,49,Latino,Mid-School,50K -- 100K,East,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,Refundable,Online,Yes,Check-In,No,0,185
2,73747291,F,42,caucasian,Grad,<25K,East,City Hotel,7/2/2015,7/6/2015,...,BB,No,No,No Deposit,Online,Yes,Check-In,No,0,119
3,67301739,M,25,African American,College,>100K,South,Airport Hotels,7/2/2015,7/3/2015,...,BB,No,No,Refundable,Agent,Yes,Check-In,Yes,5,144
4,77222321,F,62,Latino,High-School,25K --50K,East,Resort,7/3/2015,7/4/2015,...,BB,No,No,No Deposit,Direct,No,Check-In,Yes,10,242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27494,15645505,F,49,African American,Mid-School,<25K,West,City Hotel,8/31/2016,9/1/2016,...,HB,Yes,Yes,No Deposit,Online,Yes,Check-In,Yes,10,100
27495,56414525,F,49,Asian American,College,25K --50K,South,Airport Hotels,8/31/2016,9/1/2016,...,BB,No,No,Refundable,Agent,Yes,No-Show,Yes,40,194
27496,52023375,F,49,Latino,High-School,50K -- 100K,South,Airport Hotels,9/1/2016,9/2/2016,...,HB,No,No,No Deposit,Direct,Yes,Check-In,Yes,5,202
27497,98041387,M,65,African American,College,25K --50K,South,Airport Hotels,8/31/2016,9/1/2016,...,FB,No,No,No Deposit,Online,No,Canceled,Yes,20,157


In [137]:
train.dtypes

Reservation-id             int64
Gender                    object
Age                        int64
Ethnicity                 object
Educational_Level         object
Income                    object
Country_region            object
Hotel_Type                object
Expected_checkin          object
Expected_checkout         object
Booking_date              object
Adults                     int64
Children                   int64
Babies                     int64
Meal_Type                 object
Visted_Previously         object
Previous_Cancellations    object
Deposit_type              object
Booking_channel           object
Required_Car_Parking      object
Reservation_Status        object
Use_Promotion             object
Discount_Rate              int64
Room_Rate                  int64
dtype: object

In [138]:
def split_date(df,columns):
    for each in columns:
        df['day_'+each] = df[each].map(lambda x:x.day)
        df['month_'+each] = df[each].map(lambda x:x.month)
        df['year_'+each] = df[each].map(lambda x:x.year)
        df['dayofweek_'+each] = df[each].map(lambda x:x.dayofweek)


In [139]:
def date_type_conversion( df ,columns):

    for each in columns:
        for index in df.index:
                val_in = str(df.loc[index,each]).strip().split('/')
                df.loc[index,each] = '/'.join([val_in[1],val_in[0],val_in[2]])
            #print('/'.join([check_in[1],check_in[0],check_in[2]]))

        df[each] = pd.to_datetime(df[each],dayfirst=True)
    return df

def stay_period(df,
                checkin_column='Expected_checkin',
                checkout_column = 'Expected_checkout',
                stay_column = 'stay'):

    df[stay_column] = df[checkout_column] - df[checkin_column]
    return df

def booking_to_checkin(df,
                       booking_column= 'Booking_date',
                       checkin_column='Expected_checkin',
                       duration_column = 'booking_to_checkin'
                       ):
    df[duration_column] = df[checkin_column]-df[booking_column]

    return df

def total_num_participants(df,columns = ['Adults','Children','Babies'],column_name='total_participants'):
    df[column_name] = df[columns[0]]
    for each in columns[1:]:
        df[column_name] =df[column_name]+df[each]

    return df

def get_ratios(df,numerator,denominator,column_name):
    df[column_name] = df.apply(lambda x:x[numerator] if x[denominator] <= 0 else x[numerator]/x[denominator],axis = 1)
    return df

def children_to_parents(df,column_name='children_to_parent'):
    df[column_name] = df.apply(lambda x:df['Children']+df['Babies'] if x['Adults'] <= 0 else (df['Children']+df['Babies'])/x['Adults'],axis = 1)
    return df

In [ ]:
date_type_conversion(train,['Expected_checkin','Expected_checkout','Booking_date'])

In [ ]:
split_date(train,['Expected_checkin','Expected_checkout','Booking_date'])

In [ ]:
stay_period(train)
booking_to_checkin(train)
total_num_participants(train)
children_to_parents(train)

In [ ]:
train['booking_to_checkin']